In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline, Pipeline
import xgboost as xgb
import numpy as np
import joblib

In [30]:
df = pd.read_csv("/data/no_null_values.csv")

### Del duplicated values

In [31]:
df.duplicated().sum()

np.int64(21)

In [32]:
feature_cols = [col for col in df.columns if col != 'MSRP']
duplicated_features = df[df.duplicated(subset=feature_cols, keep=False)].copy()
mask_inconsistent = duplicated_features.groupby(feature_cols)['MSRP'].transform('nunique') > 1
df_inconsistent_prices = duplicated_features[mask_inconsistent].sort_values(by=feature_cols)

In [33]:
# Group by feature columns and calculate the median MSRP
# This eliminates rows with the same features but different prices,
# assigning the median price as the representative value.
df = df.groupby(feature_cols).agg(MSRP=('MSRP', 'median')).reset_index()

### Feature engineering

In [34]:
# I've created a new column categorizing cars by make. It could be done by model for more precision,
#  but I prefer to keep the simpler format.
brand_tier_mapping = {
    # Extreme Luxury brands, often associated with the highest price brackets.
    'Aston Martin': 'Extreme_Luxury',
    'Bentley': 'Extreme_Luxury',
    
    # Premium brands that cover a wide range of prices
    # but whose market position is clearly above mass-market.
    'Mercedes-Benz': 'Mid_High_Premium', 
    'BMW': 'Mid_High_Premium',
    'Audi': 'Mid_High_Premium',
    
    # Mass Market brands, typically dominating the lower and common price ranges.
    'Ford': 'Mass_Market',
    'Nissan': 'Mass_Market'
}

# Apply the mapping to the training DataFrame to create the new feature.
df['Brand_Tier'] = df['Make'].map(brand_tier_mapping)

In [35]:
df

,Make,Model,Year,Body Size,Body Style,Cylinders,Engine Aspiration,Drivetrain,Transmission,Horsepower,Torque,MSRP,Brand_Tier
0,Aston Martin,DB11,2023,Midsize,Convertible,V8,Twin-Turbo,RWD,automatic,528.0,497.0,233200.0,Extreme_Luxury
1,Aston Martin,DB11,2023,Midsize,Coupe,V8,Twin-Turbo,RWD,automatic,528.0,497.0,217000.0,Extreme_Luxury
2,Aston Martin,DBX707,2023,Large,SUV,V8,Twin-Turbo,AWD,automatic,697.0,663.0,236000.0,Extreme_Luxury
3,Aston Martin,DBX707,2024,Large,SUV,V8,Twin-Turbo,AWD,automatic,697.0,663.0,242000.0,Extreme_Luxury
4,Aston Martin,Vantage,2023,Compact,Convertible,V12,Twin-Turbo,RWD,automatic,690.0,555.0,348700.0,Extreme_Luxury
...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,Nissan,Z,2023,Compact,Coupe,V6,Twin-Turbo,RWD,automatic,400.0,350.0,50990.0,Mass_Market
740,Nissan,Z,2023,Compact,Coupe,V6,Twin-Turbo,RWD,manual,400.0,350.0,50990.0,Mass_Market
741,Nissan,Z,2024,Compact,Coupe,V6,Twin-Turbo,RWD,automatic,400.0,350.0,47310.0,Mass_Market
742,Nissan,Z,2024,Compact,Coupe,V6,Twin-Turbo,RWD,automatic,420.0,384.0,65090.0,Mass_Market


## Model 2

### Data split

In [36]:
X = df.drop(columns="MSRP")
y_log = np.log1p(df["MSRP"])
print(X.shape)
print(y_log.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

(744, 12)
(744,)


### Pipe encoding + data scaling + model

In [37]:
# 1. Define the column list
onh_cols = ["Make", "Model", "Body Style", "Cylinders", "Engine Aspiration", "Drivetrain", "Transmission", "Brand_Tier"]
ordinal_cols = ["Body Size"]
numeric_cols = ["Horsepower", "Torque"]

# 2. define the categoryes for the enconder
body_size = ["Compact", "Midsize", "Large"]

In [38]:
nominal_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ordinal_transformer = OrdinalEncoder(categories=[body_size])

In [39]:
scaler = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_cols),
        ('ohe', nominal_transformer, onh_cols),
        ('ord', ordinal_transformer, ordinal_cols)
    ],
    remainder='passthrough'
)


model = xgb.XGBRegressor(random_state=42)

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

### Train + metrics Model 2

In [ ]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9501108691034208

In [42]:
y_pred_log = pipe.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test = df.loc[y_test.index]["MSRP"]
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"R² : {r2:.4f}")
print(f"MSE (Mean Squared Error): ${mse:,.2f}")
print(f"RMSE (Root Mean Square Error):${rmse:,.2f}")
print(f"MAE (Mean Absolute Error): ${mae:,.2f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape*100:.2f}%")

R² : 0.9541
MSE (Mean Squared Error): $94,236,956.85
RMSE (Root Mean Square Error):$9,707.57
MAE (Mean Absolute Error): $4,448.44
MAPE (Mean Absolute Percentage Error): 6.12%


Thanks to the new feature and reducing the outlayers with log we archive beater results. Lets see if we can archive somethong better with GridSearcCV

### GridSearchCV Model 2

In [43]:
scorer_to_minimize = 'neg_mean_absolute_error'

param_grid = {
    'regressor__n_estimators': [500, 1000],  
    'regressor__learning_rate': [0.01, 0.05, 0.1], 
    'regressor__max_depth': [5, 7, 9],
    'regressor__min_child_weight': [1, 3],
    'regressor__colsample_bytree': [0.7, 1.0], 
}
grid_search = GridSearchCV(
    estimator=pipe,           
    param_grid=param_grid,     
    scoring=scorer_to_minimize, 
    cv=10,                     
    verbose=1,                 
    n_jobs=-1                 
)

grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 72 candidates, totalling 720 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_grid,"{'regressor__colsample_bytree': [0.7, 1.0], 'regressor__learning_rate': [0.01, 0.05, ...], 'regressor__max_depth': [5, 7, ...], 'regressor__min_child_weight': [1, 3], ...}"
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('ohe', ...), ...]"


In [ ]:
best_pipe1 = grid_search.best_estimator_

y_pred_log = best_pipe1.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_o = df.loc[y_test.index]["MSRP"]

mse = mean_squared_error(y_test_o, y_pred)
best_rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_o, y_pred)
r2 = r2_score(y_test_o, y_pred)

print("Best Model Evaluation (Optimized for Neg_MAE)")
print(f"Best parameter combination: {grid_search.best_params_}")
print(f"\nR² : {r2:.4f}")
print(f"Best Model RMSE: ${best_rmse:,.2f}")
print(f"Best MAE: {mae:.2f}")

Best Model Evaluation (Optimized for Neg_MAE)
Best parameter combination: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 5, 'regressor__min_child_weight': 1, 'regressor__n_estimators': 1000}

R² : 0.9622
Best Model RMSE Set: $8,810.43
Best MAE: 4179.45


In [45]:
# joblib.dump(grid_search, "final_model.pkl")